In [2]:
import chan_io as io
import json
import pandas as pd
import sys
import nltk
from tensorflow import keras
from tensorflow import strings
import tensorflow as tf
import re, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from datetime import datetime
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/vincent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincent/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vincent/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/vincent/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vincent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincent/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vincent/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/vincent/nltk_data...
[nltk_data]   Package omw-1.4

True

**Playground for testing analysis tools**

In [118]:
paths = ['./exampleThreads.json']
paths = io.getFiles(paths)

In [119]:
paths

['./exampleThreads.json']

In [120]:
df_posts = pd.DataFrame(columns=["timestamp","content"], index=pd.Index([],dtype=int, name="id"))

In [121]:
def parseOp(jsonPost):
    """Convert from json original post to timestamp, text tuple"""
    title = "" if jsonPost['title'] is None else jsonPost['title']
    comment = "" if jsonPost['comment'] is None else jsonPost['comment']
    return [int(jsonPost["timestamp"]), " ".join([title, comment])]

In [122]:
def parseThread(base_id, jsonThread):
    """Convert from json to pandas dataframe with columns timestamp and content and index of ids"""
    df = pd.DataFrame(columns=["timestamp","content"], index=pd.Index([],dtype=int, name="id"))
    id = base_id * 100
    op = parseOp(jsonThread["op"])
    df.loc[id] = op
    for p in jsonThread["replies"]:
        id += 1
        df.loc[id] = [op[0],p]

    return df

In [123]:
for p in paths:
    with open(p) as file:
        try:
            threads_json = json.load(file)
        except Exception as e:
            print(f"Could not read json file for threads: {e}")
            sys.exit(-1)
        
        try:
            for t in threads_json.items():
                df_posts = df_posts.append(parseThread(int(t[0]), t[1]))
        except Exception as e:
            print(f"Malformed json objects: {e}")
        

In [124]:
df_posts.head()

,timestamp,content
id,,
2485190600,1388521688,friendly reminder that making fun of dead naz...
2483806200,1388482949,What am I looking at?
2483806201,1388482949,race war now
2483806202,1388482949,>Fights corruption using knowledge\n\nWhats th...
2483806203,1388482949,Don't get your knickers in a twist about this....


In [125]:
def standardizer(text: str) -> str:
    """A method for standardizing a given string tensor"""
    lowercased = text.lower()
    html_stripped = strings.regex_replace(lowercased, "<[^>]+>", " ")
    punctuation_stripped = strings.regex_replace(html_stripped, "[%s]" % re.escape(string.punctuation), "").numpy().decode('utf-8')
    lemmatizer = WordNetLemmatizer()
    lemmatized = " ".join(map(lemmatizer.lemmatize, word_tokenize(punctuation_stripped)))
    return lemmatized

In [126]:
df_posts = df_posts.dropna()


In [127]:
df_posts.loc[:,"content"] = df_posts.loc[:,"content"].map(standardizer)

In [128]:
vocab = np.array(["hate","love","joy","happy","anger"])

In [135]:
def vectorizeLayer(vocab: np.ndarray) -> tf.Tensor:
    """
    Create a tensor for text vectorization based on the given vocabulary as a numpy array.
    If multi_hot = False, count of vocab occurrences is ignored.
    See the documentation on keras' TextVectorization layer for more
    """
    return keras.layers.TextVectorization(
        standardize=None,
        output_mode='multi_hot',
        vocabulary = vocab
    )

def mapTokenCounts(text: str, vocab: np.ndarray) -> str:
    """
    map the given post to matched words from the vocabulary
    """
    vector_layer = vectorizeLayer(vocab)
    t = tf.constant(text)
    counts = vector_layer(t).numpy()[1:]
    counts = counts.astype(np.bool8)
    return " ".join(vocab[counts])

In [136]:
df_posts.loc[:,"matched_vocab"] = df_posts.loc[:,"content"].map(lambda x: mapTokenCounts(x, vocab))

In [138]:
df_posts.loc[:,"matched_vocab"].unique()

array(['', 'happy', 'love', 'hate love', 'hate', 'hate happy', 'anger',
       'hate love joy', 'hate love happy', 'love anger',
       'hate love anger', 'hate joy', 'joy', 'love joy', 'happy anger',
       'love happy'], dtype=object)

In [3]:
vocab = io.extractVocab("./hate_vocabulary.json")

2022-01-17 19:46:01.987383: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-17 19:46:01.987522: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

